## Load WRI AqueductFloodTool Data

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Load Credentials

In [ ]:
# From the AWS Account page, copy the export scripts from the appropriate role using the "Command Line or Programmatic Access" link
# Paste the copied text into ~/credentials.env

from dotenv import dotenv_values, load_dotenv
import os
import pathlib

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

Note this references data hosted on a remote S3 bucket (at WRI)

In [ ]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config

s3_resource = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    # aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    # aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
    config=Config(signature_version=UNSIGNED)
)
bucket = s3_resource.Bucket('wri-projects')

files = []
for file in bucket.objects.filter(Prefix='AqueductFloodTool/download/v2'):
    if file.key.endswith('tif'):
        files.append(file.key)
files[0:10]

This section loads a particular innundation map thought interesting.  The main purpose is to have *SOME* data we can put into the trino layer as geospatial data.
The resampled data is stored in a notebook-local tif file...but that's not right: we want to put it somewhere into trino (possibly via [tile]DB?)

In [ ]:
from affine import Affine
import rasterio
from rasterio.enums import Resampling
from rasterio.transform import from_bounds

upscale_factor = 0.05
fp = bucket.Object('AqueductFloodTool/download/v2/inunriver_rcp8p5_00000NorESM1-M_2030_rp00002.tif').get()['Body']

def resample_raster(raster, scale=2):
    t = raster.transform

    # rescale the metadata
    transform = Affine(t.a / scale, t.b, t.c, t.d, t.e / scale, t.f)
    height = raster.height * scale
    width = raster.width * scale

    profile = raster.profile
    profile.update(transform=transform, driver='GTiff', height=height, width=width, crs=raster.crs)

    data = raster.read( # Note changed order of indexes, arrays are band, row, col order not row, col, band
            out_shape=(int(raster.count), int(height), int(width)),
            resampling=Resampling.cubic)

    return(data)

with rasterio.open(fp) as dataset:
    data=resample_raster(dataset, upscale_factor)
print("Resampling done")

arr = data[0,:]
transform = from_bounds(-180, -90, 180,90,arr.shape[1],arr.shape[0])
new_dataset = rasterio.open('/opt/app-root/src/pr-ND/WRI_data/output.tif', 'w', driver='GTiff',
                            height = arr.shape[0], 
                            width = arr.shape[1],
                            count=1, dtype=str(arr.dtype),
                            crs=dataset.crs,
                            transform=transform)

new_dataset.write(arr, 1)
new_dataset.close()

Ingest the data onto the S3 Development Bucket

In [ ]:
schemaname = 'wri'
tablename = 'resampled_geotif'
wri_plants.to_parquet('/tmp/{sname}.{tname}.parquet'.format(sname=schemaname, tname=tablename), index=False)
# Create an S3 client
s3 = boto3.client(
    service_name="s3",
    endpoint_url=os.environ['S3_DEV_ENDPOINT'],
    aws_access_key_id=os.environ['S3_DEV_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_DEV_SECRET_KEY'],
)
s3.upload_file(
    Bucket=os.environ['S3_DEV_BUCKET'],
    Key='trino/{sname}/{tname}/{tname}.parquet'.format(sname=schemaname, tname=tablename),
    Filename='/tmp/{sname}.{tname}.parquet'.format(sname=schemaname, tname=tablename)
)

Build a map and define schema mapping logic for parquet to sql

In [ ]:
_p2smap = {
    'object': 'varchar',
    'string': 'varchar',
    'float32': 'real',
    'Float32': 'real',
    'float64': 'double',
    'Float64': 'double',
    'int32': 'integer',
    'Int32': 'integer',
    'int64': 'bigint',
    'Int64': 'bigint',
    'category': 'varchar',
    'datetime64[ns, UTC]': 'timestamp',
    'datetime64[ns]': 'timestamp'
}

def pandas_type_to_sql(pt):
    st = _p2smap.get(pt)
    if st is not None:
        return st
    raise ValueError("unexpected pandas column type '{pt}'".format(pt=pt))

# add ability to specify optional dict for specific fields?
# if column name is present, use specified value?
def generate_table_schema_pairs(df):
    ptypes = [str(e) for e in df.dtypes.to_list()]
    stypes = [pandas_type_to_sql(e) for e in ptypes]
    pz = list(zip(df.columns.to_list(), stypes))
    return ",\n".join(["    {n} {t}".format(n=e[0],t=e[1]) for e in pz])

In [ ]:
import trino

conn = trino.dbapi.connect(
    host=os.environ['TRINO_HOST'],
    port=int(os.environ['TRINO_PORT']),
    user=os.environ['TRINO_USER'],
    http_scheme='https',
    auth=trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    verify=True,
)
cur = conn.cursor()

In [ ]:
# Show available schemas to ensure trino connection is set correctly
cur.execute('show schemas in osc_datacommons_dev')
cur.fetchall()

In [ ]:
cur.execute('create schema if not exists osc_datacommons_dev.wri')
cur.fetchall()

In [ ]:
cur.execute('drop table if exists osc_datacommons_dev.wri.' + tablename)
cur.fetchall()

In [ ]:
schema = generate_table_schema_pairs(wri_plants)

tabledef = """create table if not exists osc_datacommons_dev.{sname}.{tname}(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{bucket}/trino/{sname}/{tname}/'
)""".format(schema=schema,bucket=os.environ['S3_DEV_BUCKET'],sname=schemaname,tname=tablename)
print(tabledef)

# tables created externally may not show up immediately in cloud-beaver
cur.execute(tabledef)
cur.fetchall()